In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader,random_split,Dataset
import torch.optim as optim
from tqdm import tqdm
from training_utils import *

MTL Model Adapted from: https://medium.com/@aminul.huq11/multi-task-learning-a-beginners-guide-a1fc17808688

In [ ]:
seed = 43
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

torch.backends.cudnn.deterministic = True

In [ ]:
trainset = datasets.CIFAR10(root='./data/', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.CIFAR10(root='./data/', train=False, download=True, transform=transforms.ToTensor())

labels_list = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
non_animal = [0,1,8,9]
device = 'cuda'

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class NewDataset(Dataset):

    def __init__(self,data,transform=None):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        image = self.data[idx][0]
        label1 = self.data[idx][1]          #original label
        label2 = 0 if self.data[idx][1] in non_animal else 1       #animal or non-animal
        return image, label1, label2

In [ ]:

new_trainset = NewDataset(trainset,non_animal)
new_testset = NewDataset(testset,non_animal)

train_set, valid_set = random_split(new_trainset,[int(len(new_trainset)*0.9), int(len(new_trainset)*0.1)],
                                  generator=torch.Generator().manual_seed(0))

train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=100, shuffle=True)
test_loader = DataLoader(new_testset, batch_size=100, shuffle=True)

In [ ]:
class MTL_Net(nn.Module):
    def __init__(self, input_channel, num_class):
        super(MTL_Net,self).__init__()

        self.classes = num_class

        self.conv1 = nn.Conv2d(in_channels=input_channel,out_channels=8,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1)
        self.fc1 = nn.Linear(64, 256)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256,128)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, self.classes[0])
        self.fc4 = nn.Linear(128, self.classes[1])

    def forward(self, x):

        x = F.max_pool2d(F.relu(self.conv1(x)),kernel_size=3)
        x = F.max_pool2d(F.relu(self.conv2(x)),kernel_size=3)
        x = F.relu(self.fc1(x.reshape(-1,x.shape[1] * x.shape[2]*x.shape[3])))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x1 = self.fc3(x)
        x2 = self.fc4(x)

        return x1,x2

In [ ]:
class Net(nn.Module):
    def __init__(self, input_channel, num_class):
        super(Net,self).__init__()

        self.classes = num_class

        self.conv1 = nn.Conv2d(in_channels=input_channel,out_channels=8,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1)
        self.fc1 = nn.Linear(64, 256)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256,128)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, self.classes)

    def forward(self, x):

        x = F.max_pool2d(F.relu(self.conv1(x)),kernel_size=3)
        x = F.max_pool2d(F.relu(self.conv2(x)),kernel_size=3)
        x = F.relu(self.fc1(x.reshape(-1,x.shape[1] * x.shape[2]*x.shape[3])))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x1 = self.fc3(x)

        return x1

In [ ]:
num_classes = [10,2]
model = MTL_Net(3,num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)

In [ ]:
def train_model(model,trainloader,optim,criterion,epoch,device):
    model.train()
    train_loss,total,total_correct1,total_correct2 = 0,0,0,0

    for i,(inputs,tg1,tg2) in enumerate(tqdm(trainloader)):

        inputs,tg1,tg2 = inputs.to(device), tg1.to(device), tg2.to(device)
        optim.zero_grad()

        op1,op2 = model(inputs)
        loss1 = criterion(op1,tg1)
        loss2 = criterion(op2,tg2)

        total_loss =  loss1 + loss2
        total_loss.backward()

        optim.step()

        train_loss += loss1 + loss2
        _,pd1 = torch.max(op1.data,1)
        _,pd2 = torch.max(op2.data,1)

        total_correct1 += (pd1 == tg1).sum().item()
        total_correct2 += (pd2 == tg2).sum().item()

        total += tg1.size(0)

    print("Epoch: [{}]  loss: [{:.2f}] Original_task_acc [{:.2f}] animal_vs_non_animal_acc [{:.2f}]".format
                                                                          (epoch+1,train_loss/(i+1),
                                                                           (total_correct1*100/total),
                                                                          (total_correct2*100/total)))
    return train_loss/(i+1)

def train_single_model(model,trainloader,optim,criterion,epoch,device):
    model.train()
    train_loss,total,total_correct1 = 0,0,0

    for i,(inputs,tg1,tg2) in enumerate(tqdm(trainloader)):

        inputs,tg1,tg2 = inputs.to(device), tg1.to(device), tg2.to(device)
        optim.zero_grad()

        op = model(inputs)
        loss1 = criterion(op,tg1) # Change tg1 to tg2 or vice versa based on the task

        total_loss = loss1
        total_loss.backward()

        optim.step()

        train_loss += loss1.item()
        _,pd1 = torch.max(op.data,1)

        total_correct1 += (pd1 == tg1).sum().item()

        total += tg1.size(0)

    print("Epoch: [{}]  loss: [{:.2f}] Acc [{:.2f}] ".format(epoch+1,train_loss/(i+1),
                                                                           (total_correct1*100/total),
                                                                          ))
    return train_loss/(i+1)

In [ ]:
def test_model(model,testloader,optim,criterion,epoch,device):
    model.eval()
    test_loss,total,total_correct1,total_correct2 = 0,0,0,0

    with torch.no_grad():
        for i,(inputs,tg1,tg2) in enumerate(tqdm(testloader)):

            inputs,tg1,tg2 = inputs.to(device), tg1.to(device), tg2.to(device)

            op1,op2 = model(inputs)
            loss1 = criterion(op1,tg1)
            loss2 = criterion(op2,tg2)

            test_loss += loss1.item() + loss2.item()
            _,pd1 = torch.max(op1.data,1)
            _,pd2 = torch.max(op2.data,1)

            total_correct1 += (pd1 == tg1).sum().item()
            total_correct2 += (pd2 == tg2).sum().item()

            total += tg1.size(0)

    acc1 = 100. * total_correct1 / total
    acc2 = 100. * total_correct2 / total
    print("Test Epoch: [{}]  loss: [{:.2f}] Original_task_Acc [{:.2f}] animal_vs_non_animal_acc [{:.2f}]".format
                                                                          (epoch+1,test_loss/(i+1),
                                                                           acc1,acc2))

    return test_loss/(i+1), acc1, acc2

def test_single_model(model,testloader,optim,criterion,epoch,device):
    model.eval()
    test_loss,total,total_correct1,total_correct2 = 0,0,0,0

    with torch.no_grad():
        for i,(inputs,tg1,tg2) in enumerate(tqdm(testloader)):

            inputs,tg1,tg2 = inputs.to(device), tg1.to(device), tg2.to(device)

            op = model(inputs)
            loss1 = criterion(op,tg1)  # Change tg1 to tg2 or vice versa based on the task

            test_loss += loss1.item()
            _,pd1 = torch.max(op.data,1)

            total_correct1 += (pd1 == tg1).sum().item()

            total += tg1.size(0)

    acc1 = 100. * total_correct1 / total

    print("Test Epoch: [{}]  loss: [{:.2f}] Acc [{:.2f}] ".format(epoch+1,test_loss/(i+1),
                                                                           acc1))

    return test_loss/(i+1), acc1

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):

    _ = train_model(model,train_loader,optimizer,criterion,epoch,device)
    _,_,_ = test_model(model,valid_loader,optimizer,criterion,epoch,device)

100%|██████████| 450/450 [00:22<00:00, 20.00it/s]


Epoch: [1]  loss: [2.98] Original_task_acc [10.07] animal_vs_non_animal_acc [58.23]


100%|██████████| 50/50 [00:02<00:00, 24.30it/s]


Test Epoch: [1]  loss: [2.97] Original_task_Acc [10.08] animal_vs_non_animal_acc [60.90]


100%|██████████| 450/450 [00:23<00:00, 18.88it/s]


...

Test Epoch: [49]  loss: [1.71] Original_task_Acc [48.46] animal_vs_non_animal_acc [89.78]


100%|██████████| 450/450 [00:20<00:00, 21.68it/s]


Epoch: [50]  loss: [1.67] Original_task_acc [47.83] animal_vs_non_animal_acc [90.57]


100%|██████████| 50/50 [00:01<00:00, 25.23it/s]

Test Epoch: [50]  loss: [1.70] Original_task_Acc [48.64] animal_vs_non_animal_acc [88.72]


In [ ]:
_,_,_ = test_model(model,test_loader,optimizer,criterion,epoch,device)

In [ ]:
num_classes = 10
model = Net(3,num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):

    _ = train_single_model(model,train_loader,optimizer,criterion,epoch,device)
    _,_ = test_single_model(model,valid_loader,optimizer,criterion,epoch,device)

In [ ]:

_,_ = test_single_model(model,test_loader,optimizer,criterion,epoch,device)

# AIHWKit

In [ ]:
!wget https://aihwkit-gpu-demo.s3.us-east.cloud-object-storage.appdomain.cloud/aihwkit-0.9.1+cuda117-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install aihwkit-0.9.1+cuda117-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

--2024-12-18 15:34:42--  https://aihwkit-gpu-demo.s3.us-east.cloud-object-storage.appdomain.cloud/aihwkit-0.9.1+cuda117-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Resolving aihwkit-gpu-demo.s3.us-east.cloud-object-storage.appdomain.cloud (aihwkit-gpu-demo.s3.us-east.cloud-object-storage.appdomain.cloud)... 169.63.118.98
Connecting to aihwkit-gpu-demo.s3.us-east.cloud-object-storage.appdomain.cloud (aihwkit-gpu-demo.s3.us-east.cloud-object-storage.appdomain.cloud)|169.63.118.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 379098522 (362M) [application/octet-stream]
Saving to: ‘aihwkit-0.9.1+cuda117-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.1’

aihwkit-0.9.1+cuda1 100%[===================>] 361.54M  45.8MB/s    in 8.4s    

2024-12-18 15:34:51 (43.3 MB/s) - ‘aihwkit-0.9.1+cuda117-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.1’ saved [379098522/379098522]

Processing ./aihwkit-0.9.1+cuda117-cp310-cp310-manylinux_2

In [ ]:
from aihwkit.nn import AnalogConv2d, AnalogLinear
from aihwkit.simulator.configs import InferenceRPUConfig
from aihwkit.simulator.noise_models import PCMLikeNoiseModel
import torch.nn.functional as F
import torch.nn as nn

/usr/local/lib/python3.10/dist-packages/aihwkit/simulator/noise_models.py:18: FutureWarning: 

The `aihwkit.simulator.noise_models` module has been superseded by the `aihwkit.inference` module. Please replace `from aihwkit.simulator.noise_models import ...` with `from aihwkit.inference import ...` in your import statement.

  warnings.warn(


In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader,random_split,Dataset
import torch.optim as optim
from tqdm import tqdm

In [ ]:
def create_layer(layer_type, analog, *args, **kwargs):
    """
    Dynamically creates a layer as analog or digital.

    Parameters:
    - layer_type: str, 'conv' for convolutional layers, 'linear' for fully connected layers.
    - analog: bool, whether the layer should be analog.
    - *args, **kwargs: additional arguments for layer initialization.

    Returns:
    - A layer (AnalogConv2d, AnalogLinear, nn.Conv2d, or nn.Linear).
    """
    if analog:
        rpu_config = InferenceRPUConfig()
        rpu_config.noise_model = PCMLikeNoiseModel()  # Simulate PCM-like noise
        rpu_config.forward.out_res = 1 / 128.0  # Simulate limited precision

        if layer_type == 'conv':
            return AnalogConv2d(*args, **kwargs, rpu_config=rpu_config)
        elif layer_type == 'linear':
            return AnalogLinear(*args, **kwargs, rpu_config=rpu_config)
    else:
        if layer_type == 'conv':
            return nn.Conv2d(*args, **kwargs)
        elif layer_type == 'linear':
            return nn.Linear(*args, **kwargs)

    raise ValueError("Invalid layer_type. Use 'conv' or 'linear'.")

In [ ]:
class DynamicLayer(nn.Module):
    def __init__(self, layer_type, *args, **kwargs):
        """
        Dynamic Layer Wrapper for analog/digital layers.

        Parameters:
        - layer_type: str, either 'conv' or 'linear'.
        - *args, **kwargs: Additional arguments for layer initialization.
        """
        super(DynamicLayer, self).__init__()

        self.layer_type = layer_type
        self.args = args
        self.kwargs = kwargs

        # Initialize analog and digital layers
        rpu_config = InferenceRPUConfig()
        rpu_config.noise_model = PCMLikeNoiseModel()  # Simulate PCM-like noise
        rpu_config.forward.out_res = 1 / 128.0  # Simulate limited precision

        if layer_type == 'conv':
            self.analog_layer = AnalogConv2d(*args, **kwargs, rpu_config=rpu_config)
            self.digital_layer = nn.Conv2d(*args, **kwargs)
        elif layer_type == 'linear':
            self.analog_layer = AnalogLinear(*args, **kwargs, rpu_config=rpu_config)
            self.digital_layer = nn.Linear(*args, **kwargs)
        else:
            raise ValueError("Invalid layer_type. Use 'conv' or 'linear'.")

        # Start with digital implementation
        self.is_analog = False

    def forward(self, x):
        if self.is_analog:
            return self.analog_layer(x)
        else:
            return self.digital_layer(x)

    def toggle(self, is_analog):
        """
        Switch between analog and digital implementation.

        Parameters:
        - is_analog: bool, whether to use the analog layer.
        """
        self.is_analog = is_analog

        # If switching to analog, synchronize weights
        if self.is_analog:
            if self.layer_type == 'conv':
                self.analog_layer.set_weights(self.digital_layer.weight.detach().cpu().numpy().reshape(-1))
            elif self.layer_type == 'linear':
                self.analog_layer.set_weights(self.digital_layer.weight.detach().cpu().numpy(),
                                               self.digital_layer.bias.detach().cpu().numpy())

        # If switching to digital, synchronize weights
        else:
            if self.layer_type == 'conv':
                weights = torch.tensor(self.analog_layer.get_weights()[0]).reshape(self.digital_layer.weight.shape)
                self.digital_layer.weight.data.copy_(weights)
            elif self.layer_type == 'linear':
                weights, bias = self.analog_layer.get_weights()
                self.digital_layer.weight.data.copy_(torch.tensor(weights))
                self.digital_layer.bias.data.copy_(torch.tensor(bias))

In [ ]:
class MTL_Net_DynamicToggle(nn.Module):
    def __init__(self, input_channel, num_class):
        super(MTL_Net_DynamicToggle, self).__init__()
        self.classes = num_class

        # Replace layers with dynamic layers
        self.conv1 = DynamicLayer('conv', in_channels=input_channel, out_channels=8, kernel_size=3, stride=1)
        self.conv2 = DynamicLayer('conv', in_channels=8, out_channels=16, kernel_size=3, stride=1)
        self.fc1 = DynamicLayer('linear', 64, 256)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = DynamicLayer('linear', 256, 128)
        self.dropout2 = nn.Dropout(0.3)

        # Task-specific layers remain digital
        self.fc3 = nn.Linear(128, self.classes[0])
        self.fc4 = nn.Linear(128, self.classes[1])

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=3)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=3)
        x = F.relu(self.fc1(x.reshape(-1, x.shape[1] * x.shape[2] * x.shape[3])))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x1 = self.fc3(x)  # Task 1 output
        x2 = self.fc4(x)  # Task 2 output

        return x1, x2

    def toggle_layer(self, layer_name, is_analog):
        """
        Toggle a specific layer between analog and digital.
        """
        getattr(self, layer_name).toggle(is_analog)


In [ ]:
def train_model_dynamic(model, trainloader, optim, criterion, epoch, device, toggle_config=None):
    """
    Train function with dynamic analog/digital toggling.

    Parameters:
    - toggle_config: dict, specifies which layers to toggle and when during training.
                     Example: {'conv1': True, 'fc1': False}.
    """
    model.train()  # Ensure the model is in training mode
    train_loss, total, total_correct1, total_correct2 = 0, 0, 0, 0

    # Apply toggling at the start of training (if toggle_config is provided)
    if toggle_config:
        for layer_name, is_analog in toggle_config.items():
            model.toggle_layer(layer_name, is_analog)

    for i, (inputs, tg1, tg2) in enumerate(tqdm(trainloader)):
        inputs, tg1, tg2 = inputs.to(device), tg1.to(device), tg2.to(device)
        optim.zero_grad()

        # Forward pass through the model
        op1, op2 = model(inputs)

        # Compute losses for both tasks
        loss1 = criterion(op1, tg1)
        loss2 = criterion(op2, tg2)
        total_loss = loss1 + loss2

        # Backpropagation
        total_loss.backward()

        # Update weights
        optim.step()

        # Accumulate metrics
        train_loss += total_loss.item()
        _, pd1 = torch.max(op1.data, 1)
        _, pd2 = torch.max(op2.data, 1)

        total_correct1 += (pd1 == tg1).sum().item()
        total_correct2 += (pd2 == tg2).sum().item()
        total += tg1.size(0)

    # Compute and log metrics
    print("Epoch: [{}]  loss: [{:.2f}] Original_task_acc [{:.2f}] animal_vs_non_animal_acc [{:.2f}]".format(
        epoch + 1, train_loss / (i + 1),
        (total_correct1 * 100 / total),
        (total_correct2 * 100 / total)
    ))

    return train_loss / (i + 1)




In [ ]:
def test_model_dynamic(model, testloader, criterion, epoch, device, toggle_config=None):
    """
    Test function with dynamic analog/digital toggling.

    Parameters:
    - toggle_config: dict, specifies which layers to toggle during testing.
                     Example: {'conv1': True, 'fc1': False}.
    """
    model.eval()  # Ensure the model is in evaluation mode
    test_loss, total, total_correct1, total_correct2 = 0, 0, 0, 0

    # Apply toggling at the start of testing (if toggle_config is provided)
    if toggle_config:
        for layer_name, is_analog in toggle_config.items():
            model.toggle_layer(layer_name, is_analog)

    with torch.no_grad():
        for i, (inputs, tg1, tg2) in enumerate(tqdm(testloader)):
            inputs, tg1, tg2 = inputs.to(device), tg1.to(device), tg2.to(device)

            # Forward pass through the model
            op1, op2 = model(inputs)

            # Compute losses for both tasks
            loss1 = criterion(op1, tg1)
            loss2 = criterion(op2, tg2)

            # Accumulate metrics
            test_loss += loss1.item() + loss2.item()
            _, pd1 = torch.max(op1.data, 1)
            _, pd2 = torch.max(op2.data, 1)

            total_correct1 += (pd1 == tg1).sum().item()
            total_correct2 += (pd2 == tg2).sum().item()
            total += tg1.size(0)

    # Compute accuracies
    acc1 = 100. * total_correct1 / total
    acc2 = 100. * total_correct2 / total

    # Log metrics
    print("Test Epoch: [{}]  loss: [{:.2f}] Original_task_Acc [{:.2f}] animal_vs_non_animal_acc [{:.2f}]".format(
        epoch + 1, test_loss / (i + 1), acc1, acc2
    ))

    return test_loss / (i + 1), acc1, acc2


In [ ]:
model = MTL_Net_DynamicToggle(input_channel=3, num_class=[10, 5]).to('cuda')

model.toggle_layer('conv1', is_analog=False)
model.toggle_layer('conv2', is_analog=False)
model.toggle_layer('fc1', is_analog=False)
model.toggle_layer('fc2', is_analog=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

toggle_config_train = {
    'conv1': False,  # Switch conv1 to analog
    'fc1': False    # Keep fc1 as digital
}

toggle_config_test = {
    'conv1': True,  # Switch conv1 to digital during testing
    'fc1': True      # Switch fc1 to analog during testing
}

for epoch in range(50):
    # Dynamically toggle during training
    train_loss = train_model_dynamic(model, train_loader, optimizer, criterion, epoch, device='cuda', toggle_config=toggle_config_train)

    # Dynamically toggle during testing
    test_loss, acc1, acc2 = test_model_dynamic(model, test_loader, criterion, epoch, device='cuda', toggle_config=toggle_config_test)


<ipython-input-4-1accb224cb85>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weights = torch.tensor(self.analog_layer.get_weights()[0]).reshape(self.digital_layer.weight.shape)
<ipython-input-4-1accb224cb85>:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.digital_layer.weight.data.copy_(torch.tensor(weights))
<ipython-input-4-1accb224cb85>:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.digital_layer.bias.data.copy_(torch.tensor(bias))
100%|██████████| 450/450 [00:19<00:00, 22.81it/s]


Epoch: [1]  loss: [2.54] Original_task_acc [20.89] animal_vs_non_animal_acc [76.61]


100%|██████████| 100/100 [00:04<00:00, 21.25it/s]


Test Epoch: [1]  loss: [3.11] Original_task_Acc [17.34] animal_vs_non_animal_acc [60.23]


100%|██████████| 450/450 [00:20<00:00, 22.18it/s]


...

Test Epoch: [49]  loss: [4.94] Original_task_Acc [18.48] animal_vs_non_animal_acc [45.13]


100%|██████████| 450/450 [00:21<00:00, 20.69it/s]


Epoch: [50]  loss: [1.43] Original_task_acc [55.57] animal_vs_non_animal_acc [92.01]


100%|██████████| 100/100 [00:04<00:00, 23.78it/s]

Test Epoch: [50]  loss: [5.27] Original_task_Acc [17.82] animal_vs_non_animal_acc [44.61]
